In [ ]:
%pylab
%matplotlib inline

In [ ]:
%run pdev notebook

# Radiosonde SONDE

In [ ]:
ident = "SONDE"
plt.rcParams['figure.figsize'] = [12.0, 6.0]
plt.rcParams['lines.linewidth'] = 2
plt.rcParams['font.size'] = 15
yplevs = np.array([10,100,200,300,400,500,700,925])*100
save = True

In [ ]:
!mkdir -p figures

In [ ]:
rt.load_config()
rt.config

In [ ]:
isonde = rt.cls.Radiosonde(ident)

In [ ]:
#
# All the data available
#
isonde.list_store()

## Load Data

In [ ]:
# close=False -> stay on disk, 
#      =True  -> load to memory
close = False

### ERA5

In [ ]:
if False:
    isonde.add('ERA5', filename='ERA5_*.nc', cfunits=True, close=close, verbose=1)
if False:
    isonde.add('ERA5_meta', filename='*_ERA5_station.nc', cfunits=True, close=close, verbose=1)

### ERA Interim

In [ ]:
if False:
    isonde.add('ERAI', filename='ERAI_*.nc', cfunits=True, close=close, verbose=1)

### IGRA v2

In [ ]:
if False:
    isonde.add('IGRAv2', cfunits=True, close=close, verbose=1)

### Upper Air Database (UADB)

In [ ]:
if False:
    isonde.add('UADB', cfunits=True, close=close, verbose=1)

### JRA-55

In [ ]:
if False:
    isonde.add('JRA55', close=close, verbose=1)

### CERA-20C

In [ ]:
if False:
    isonde.add('CERA20C', close=close, verbose=1)

### Standardized Combined Data

In [ ]:
idata = None
#
# ERA5
#
if isonde.in_store('dataE5JC'):
    isonde.add('dataE5JC', verbose=1)
    idata = isonde.data.dataE5JC
#
# ERA Interim
#
if isonde.in_store('dataEIJC') and idata is None:
    isonde.add('dataEIJC', verbose=1)
    idata = isonde.data.dataEIJC
#
# IGRA 
#
if isonde.in_store('dataIE5JC') and idata is None:
    isonde.add('dataIE5JC', verbose=1)
    idata = isonde.data.dataIE5JC

### Experiment Data

In [ ]:
isonde.list_store(pattern='exp')

In [ ]:
ivar = 'dpd'
version = 'v1'
isrc = 'mars5'
ires = 'era5'
expdata = None
#
# ERA5
#
if isonde.in_store('exp{}{}_{}_{}.nc'.format(ivar,version,isrc,ires)):
    isonde.add('exp{}{}_{}_{}'.format(ivar,version,isrc,ires), verbose=1)
    expdata = isonde.data['exp{}{}_{}_{}'.format(ivar,version,isrc,ires)]
#
# ERA Interim
#
if expdata is None:
    isrc = 'marsi'
    ires = 'erai'
    if isonde.in_store('exp{}{}_{}_{}.nc'.format(ivar,version,isrc,ires)):
        isonde.add('exp{}{}_{}_{}'.format(ivar,version,isrc,ires), verbose=1)
        expdata = isonde.data['exp{}{}_{}_{}'.format(ivar,version,isrc,ires)]
#
# JRA55
#
if expdata is None:
    isrc = 'mars5'
    ires = 'jra55'
    if isonde.in_store('exp{}{}_{}_{}.nc'.format(ivar,version,isrc,ires)):
        isonde.add('exp{}{}_{}_{}'.format(ivar,version,isrc,ires), verbose=1)
        expdata = isonde.data['exp{}{}_{}_{}'.format(ivar,version,isrc,ires)]

In [ ]:
if idata is None:
    print("No data ?")
    exit()

In [ ]:
#
# Some definitions
#
times = [0, 12]
start = '1979'
ende = '2019'
period = slice(start, ende)
period_str = "%s-%s" % (start, ende)
#
# Subset to only that period
#
idata = idata.sel(time=period, hour=times)

## Station Map

In [ ]:
rt.plot.map.station_class(isonde, states=True,  rivers=True, land=True, lakes=True)
if save:
    savefig('figures/%s_station.png' % ident)

# Data Availability

In [ ]:
dpdvars = []
tvars = []
for jvar in list(idata.data_vars):
    if 'dpd_' in jvar:
        if not any([i in jvar for i in ['err','_fg_','snht']]):
            dpdvars.append(jvar)
    if 't_' in jvar:
        if not any([i in jvar for i in ['err','_fg_','snht']]):
            tvars.append(jvar)
print(dpdvars)
print(tvars)

## Dewpoint depression

In [ ]:
counts = idata.reset_coords()[dpdvars].count('time').sum('hour').to_dataframe()
counts.index /= 100.
counts.plot()
xticks(yplevs/100)
grid()
title("%s Counts %s" % (ident, period_str))
ylabel("Total counts [1]")
if save:
    savefig('figures/%s_dpd_counts.png' % ident)

## Temperature

In [ ]:
counts = idata.reset_coords()[tvars].count('time').sum('hour').to_dataframe()
counts.index /= 100.
counts.plot()
xticks(yplevs/100)
grid()
title("%s Counts %s" % (ident, period_str))
ylabel("Total counts [1]")
if save:
    savefig('figures/%s_t_counts.png' % ident)

## Annual

In [ ]:
counts = idata.reset_coords()[dpdvars].count('plev').resample(time='A').sum().to_dataframe()
n = len(idata.hour.values)
f, ax = subplots(n,1, sharex=True)
ax[0].set_title("%s Annual counts %s" % (ident, period_str))
for i,ihour in enumerate(idata.hour.values):
    counts.xs(ihour, level=0).plot(grid=True, ax=ax[i], legend=True if i==0 else False)
    ax[i].set_ylabel("%02d Z" % (ihour))
ax[i].set_xlabel('Years')
tight_layout()
if save:
    savefig('figures/%s_dpd_ancounts.png' % (ident))

In [ ]:
counts = idata.reset_coords()[tvars].count('plev').resample(time='A').sum().to_dataframe()
n = len(idata.hour.values)
f, ax = subplots(n,1, sharex=True)
ax[0].set_title("%s Annual counts %s" % (ident, period_str))
for i,ihour in enumerate(idata.hour.values):
    counts.xs(ihour, level=0).plot(grid=True, ax=ax[i], legend=True if i==0 else False)
    ax[i].set_ylabel("%02d Z" % (ihour))
ax[i].set_xlabel('Years')
tight_layout()
if save:
    savefig('figures/%s_t_ancounts.png' % (ident))

# Dewpoint depression

In [ ]:
obs = 'dpd_{}'.format(isrc)
hdim = 'hour'
for ihour in idata[hdim].values:
    rt.plot.time.var(idata[obs].sel(**{hdim:ihour}), dim='time', lev='plev', 
                     title='%s %s Radiosonde at %02d Z' % (ident, obs, ihour))

# Temperature

In [ ]:
obs = 't_{}'.format(isrc)
hdim = 'hour'
for ihour in idata[hdim].values:
    rt.plot.time.var(idata[obs].sel(**{hdim:ihour}), dim='time', lev='plev', 
                     title='%s %s Radiosonde at %02d Z' % (ident, obs, ihour))

# Comparison with Reanalysis

In [ ]:
dim = 'time'
hdim = 'hour'
lev = 'plev'
ivar = 'dpd'
obs = '{}_{}'.format(ivar, isrc)
plotvars = []
#
# Select Variables
#
for jvar in list(idata.data_vars):
    if '_' in jvar:
        iname = jvar.split('_')[1]
        if jvar == "%s_%s" %(ivar, iname):
            plotvars += [jvar]

print(plotvars)
#
# Select Level
#
ipres=10000
#
# Plot
#
ylims = (np.round(idata[obs].min()), np.round(idata[obs].max()))
for i,j in idata[plotvars].groupby(hdim):
    m = j.sel(**{lev:ipres}).resample(**{dim:'M'}).mean(dim)
    f, ax = plt.subplots(figsize=(16,4))
    for jvar in plotvars:
        rt.plot.time.var(m[jvar], ax=ax, dim=dim, label=jvar.replace(ivar+'_',''))
    ax.set_ylabel("%s [%s]" % (ivar, idata[jvar].attrs['units']))
    ax.set_xlabel('Time [M]')
    ax.set_title('%s %s Comparison %s %02dZ at %d hPa' %(ident, ivar, period_str, i, ipres/100))
    ax.legend(ncol=len(plotvars))
    ax.set_ylim(ylims)
    tight_layout()
    if save:
        savefig('figures/%s_%s_comparison_%04d_%02dZ.png' % (ident, ivar, ipres/100, i))

## Departures

In [ ]:
dim = 'time'
hdim = 'hour'
lev = 'plev'
ivar = 'dpd'
obs = '{}_{}'.format(ivar, isrc)
plotvars = []
#
# Select Variables
#
for jvar in list(idata.data_vars):
    if '_' in jvar:
        iname = jvar.split('_')[1]
        if jvar == "%s_%s" %(ivar, iname):
            plotvars += [jvar]

print(plotvars)
#
# Select Level
#
ipres=30000
#
# Plot
#
ylims = (-10,10)  # Manual
for i,j in idata[plotvars].groupby(hdim):
    m = j.sel(**{lev:ipres}).resample(**{dim:'M'}).mean(dim)
    f, ax = plt.subplots(figsize=(16,4))
    for jvar in plotvars:
        if jvar == obs:
            continue
        rt.plot.time.var(m[obs] - m[jvar], ax=ax, dim=dim, label=jvar.replace(ivar+'_',''))
    ax.set_ylabel("%s [%s]" % (ivar, idata[jvar].attrs['units']))
    ax.set_xlabel('Time [M]')
    ax.set_title('%s Departures %s (OBS-BG) %s %02dZ at %d hPa' %(ident, ivar, period_str, i, ipres/100))
    ax.legend(ncol=len(plotvars))
    ax.set_ylim(ylims)
    tight_layout()
    if save:
        savefig('figures/%s_%s_dep_%04d_%02dZ.png' % (ident, ivar, ipres/100, i))

# Adjustment Process

In [ ]:
if expdata is None:
    #
    # Make Experiments 
    #
    expdata = idata.copy()
else:
    expdata = expdata.sel(**{dim: period})

## SNHT

In [ ]:
dim = 'time'
hdim = 'hour'
ivar = 'dpd'
obs = '{}_{}'.format(ivar, isrc)
res = '{}_{}'.format(ivar, ires)
#
# Execute SNHT ?
#
if not '{}_snht'.format(obs) in expdata.data_vars:
    #
    # Calculate SNHT values with Parameters (window and missing)
    #
    expdata = rt.bp.snht(expdata, var=obs, dep=res, dim=dim, 
                         window=1460, 
                         missing=600, 
                         verbose=1)
    #
    # Apply Threshold (threshold) and detect Peaks
    # allowed distances between peaks (dist)
    # minimum requires significant levels (min_levels)
    #
    expdata = expdata.groupby(hdim).apply(rt.bp.apply_threshold,
                                          threshold=50,
                                          dist=730,
                                          min_levels=3,
                                          var=obs + '_snht',
                                          dim=dim)

In [ ]:
#
# Plot SNHT
#
for i,j in expdata.groupby(hdim):
    ax = rt.plot.time.threshold(j[obs + '_snht'], dim=dim, lev=lev, logy=False, 
                                title=" %s SNHT %s at %02dZ" % (ident, period_str, i), 
                                figsize=(12,4), 
                                yticklabels=yplevs)
    rt.plot.time.breakpoints(j[obs + '_snht_breaks'], ax=ax, startend=True)
    tight_layout()
    if save:
        savefig('figures/%s_%s_snht_%s_%02dZ.png' % (ident, obs, ires, i))

## Breakpoints

In [ ]:
#
# Give Breakpoint Information 
#
for i,j in expdata.groupby(hdim):
    _=rt.bp.get_breakpoints(j[obs + '_snht_breaks'], dim=dim, verbose=1)

## Adjustments

In [ ]:
dim = 'time'
hdim = 'hour'
ivar = 'dpd'
obs = '{}_{}'.format(ivar, isrc)
res = '{}_{}'.format(ivar, ires)

# plotvars = [i for i in expdata.data_vars if '_dep' in i]
adjvars = "{obs},{obs}_m,{obs}_q,{obs}_qa".format(obs=obs)
adjvars = adjvars.split(',')
print(adjvars)

In [ ]:
missing = False
for jvar in adjvars:
    if jvar not in expdata.data_vars:
        missing = True


### Run standard adjustment process

In [ ]:
if missing:
    from detect import run_standard
    expdata = run_standard(idata, obs, res, meanadj=True, qadj=True, qqadj=True, verbose=1)

## Breakpoint Stats

In [ ]:
ipres=85000
#
# MEAN ADJ
#
bins = np.round(np.nanpercentile(np.ravel(expdata[obs].sel(**{lev:ipres}).values), [1,99]))
bins = np.arange(bins[0]-2,bins[1]+2,1)
for i,j in expdata.groupby(hdim):
    rt.plot.breakpoints_histograms(j.sel(**{lev:ipres}), 
                                   obs, '{}_m'.format(obs), '{}_snht_breaks'.format(obs),
                                   figsize=(18,8), 
                                   other_var=res,
                                   bins=bins);
    if save:
        savefig('figures/%s_bhist_m_%s_%02dZ_%04dhPa.png' % (ident, ivar, i, ipres/100))

In [ ]:
ipres=85000
#
# QUANTIL ADJ
#
bins = np.round(np.nanpercentile(np.ravel(expdata[obs].sel(**{lev:ipres}).values), [1,99]))
bins = np.arange(bins[0]-2,bins[1]+2,1)
for i,j in expdata.groupby(hdim):
    rt.plot.breakpoints_histograms(j.sel(**{lev:ipres}), 
                                   obs, '{}_q'.format(obs), '{}_snht_breaks'.format(obs),
                                   figsize=(18,8), 
                                   other_var=res,
                                   bins=bins);
    if save:
        savefig('figures/%s_bhist_q_%s_%02dZ_%04dhPa.png' % (ident, ivar, i, ipres/100))

In [ ]:
ipres=85000
#
# QUANTIL ADJ
#
bins = np.round(np.nanpercentile(np.ravel(expdata[obs].sel(**{lev:ipres}).values), [1,99]))
bins = np.arange(bins[0]-2,bins[1]+2,1)
for i,j in expdata.groupby(hdim):
    rt.plot.breakpoints_histograms(j.sel(**{lev:ipres}), 
                                   obs, '{}_qa'.format(obs), '{}_snht_breaks'.format(obs),
                                   figsize=(18,8), 
                                   other_var=res,
                                   bins=bins);
    if save:
        savefig('figures/%s_bhist_qa_%s_%02dZ_%04dhPa.png' % (ident, ivar, i, ipres/100))

## Adjustment methods

In [ ]:
bvar = '{}_snht_breaks'.format(obs)
#
# Select Level
#
ipres=30000
#
# Plot
#
ylims = np.round(np.nanpercentile(np.ravel(expdata[obs].sel(**{lev:ipres}).rolling(**{dim:30, 'center':True, 'min_periods':10}).mean().values), [1,99]))
ylims += [-2,2]
for i,j in expdata[adjvars].groupby(hdim):
    m = j.sel(**{lev:ipres}).rolling(**{dim:30, 'center':True, 'min_periods':10}).mean()
    f, ax = plt.subplots(figsize=(16,4))
    for jvar in adjvars:
        rt.plot.time.var(m[jvar], ax=ax, dim=dim, label=jvar[-1:].upper() if jvar != obs else ivar,  ls='-' if jvar == obs else '--')
    if bvar in expdata.data_vars:
        rt.plot.time.breakpoints(expdata[bvar].sel(**{hdim:i}), ax=ax, color='k', lw=2, ls='--')    
    ax.set_ylabel("%s [%s]" % (ivar, expdata[jvar].attrs['units']))
    ax.set_xlabel('Time [M]')
    ax.set_title('%s Adjustments %s %s %02dZ at %d hPa' %(ident, ivar, period_str, i, ipres/100))
    ax.legend(ncol=len(plotvars))
    ax.set_ylim(ylims)
    tight_layout()
    if save:
        savefig('figures/%s_%s_adj_%04d_%02dZ.png' % (ident, ivar, ipres/100, i))    
    

# Analysis

In [ ]:
#
# Monthly Means
#
variables = list(unique(dpdvars + tvars + adjvars))
for jvar in variables[:]:
    if jvar not in expdata.data_vars:
        variables.remove(jvar)
print(variables)
mdata = expdata[variables].resample(**{dim:'M'}).mean(keep_attrs=True)

## Trends

In [ ]:
trends = rt.met.time.trend(mdata, period=period, dim=dim, only_slopes=True)
with xr.set_options(keep_attrs=True):
    trends = trends*3650.  # Trends per Decade
for jvar in trends.data_vars:
    trends[jvar].attrs['units'] = trends[jvar].attrs['units'].replace('day','decade')

In [ ]:
xlims = (np.round(trends.min().to_array().min()), np.round(trends.max().to_array().max()))
n = mdata[hdim].size
f,ax = rt.plot.init_fig_horizontal(n=n, ratios=tuple([2]*n), sharey=True)
for i, ihour in enumerate(trends[hdim].values):
    for jvar in variables:
        rt.plot.profile.var(trends[jvar].sel(**{hdim:ihour}), ax=ax[i], label=jvar[-1:].upper() if jvar != obs else ivar)
    ax[i].set_title('%02d' % ihour)
    ax[i].set_xlim(xlims)
    ax[i].set_xlabel("%s [%s]" % (mdata[obs].attrs['standard_name'], trends[jvar].attrs['units']))
    
f.suptitle('%s %s Trends %s' % (ident, ivar.upper(), period_str))
if save:
    savefig('figures/%s_trends_%s.png' % (ident, ivar))

## Statistics

In [ ]:
from detect import skills_table

In [ ]:
for jvar in mdata.data_vars:
    if jvar == obs or jvar == res:
        continue
    _ , ytable = skills_table(mdata[obs], mdata[res], mdata[jvar])
    print("#"*50)
    print(ident, obs, res, jvar)
    print(ytable)
    print("#"*50)